# Реализовать разложение по сингулярным значениям (SVD) для ленточных матриц.
## создание произвольной ленточной матрицы.
На вход подаётся массив $n \times m$ - в таком виде хранятся ленточные матрицы.
$n$ - размер матрицы, $m$ - ширина ленты, нечётное число.
<img src='Matrix_storage-2.png'></img>

In [1]:
import numpy as np

In [2]:
def random_band(n, m, seed=1234):
    '''
    создаёт случайную ленту, на основе которой будет создана ленточная матрица
    n - длина ленты
    m - ширина ленты
    '''
    band = np.zeros((n, m))
    l = (m - 1) // 2 #полуширина ленты
    for j in range(l, -1, -1):
        band[j:, l - j] = np.random.rand(n - j)
    for j in range(1, l + 1):
        band[:n - j, l + j] = np.random.rand(n - j)
    return band


A = random_band(5, 5)
print(A)

[[0.         0.         0.41146889 0.11653018 0.44631611]
 [0.         0.62776341 0.79809809 0.88558927 0.07640795]
 [0.51069872 0.41430726 0.92344932 0.06246516 0.5314376 ]
 [0.03904595 0.65727217 0.51163805 0.28351214 0.        ]
 [0.51058574 0.10611292 0.68285181 0.         0.        ]]


In [3]:
def band_matrix(band):
    '''
    создаёт ленточную матрицу из ленты band
    крайние элементы столбцов ленты должны содержать нули согласно рисунку выше
    '''
    n, m = band.shape
    l = (m - 1) // 2
    matrix = np.zeros((n, n))
    
    i = 0
    for j in range(l, -1, -1):
        matrix[i, :i + l + 1] = band[i, j:]
        i += 1
    start = i
    i = n - 1
    for j in range(l, 0, -1):
        matrix[i, i - l:] = band[i, : -j]
        i -= 1
    stop = i
    for i in range(start, stop + 1):
        matrix[i, i - l : i + l + 1] = band[i]
    return matrix

In [4]:
A = band_matrix(A)
print(A)

[[0.41146889 0.11653018 0.44631611 0.         0.        ]
 [0.62776341 0.79809809 0.88558927 0.07640795 0.        ]
 [0.51069872 0.41430726 0.92344932 0.06246516 0.5314376 ]
 [0.         0.03904595 0.65727217 0.51163805 0.28351214]
 [0.         0.         0.51058574 0.10611292 0.68285181]]


<img src='one_way-2.png'></img>

In [5]:
A2 = A.T @ A
print(A2)

[[0.82420674 0.76055152 1.21119013 0.07986699 0.27140451]
 [0.76055152 0.82371494 1.16705198 0.1068382  0.23124846]
 [1.21119013 1.16705198 2.52892957 0.51581466 1.02575473]
 [0.07986699 0.1068382  0.51581466 0.28277352 0.25071133]
 [0.27140451 0.23124846 1.02575473 0.25071133 0.82909166]]


In [6]:
#спектральное разложение w - собственные значения, v - собственные вектора
w, v = np.linalg.eig(A2)
n = A2.shape[0]
#заполнения лямда собственными значениями
L = np.zeros((n, n))
for i in range(n):
    L[i, i] = w[i]
#проверка правильности разложения
np.allclose(A2, v @ L @ np.linalg.inv(v))

True

In [7]:
# находим сумму
Sigma = np.sqrt(L)

In [8]:
#находим матрицу u
u = A @ v @ np.linalg.inv(Sigma)

In [9]:
#проверка правильности
np.allclose(A, u @ Sigma @ np.linalg.inv(v))

True

In [10]:
#нахождение u и суммы встроенным методом
u_qr, Sigma_qr = np.linalg.qr(A @ v)

In [11]:
#проверка правильности
np.allclose(A, u_qr @ Sigma_qr @ np.linalg.inv(v))

True

In [12]:
#проверка правильности
np.allclose(A, u_qr @ Sigma_qr @ v.T)

True